In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from collections import defaultdict 

import csv
from tqdm import tqdm
from pathlib import Path 

In [2]:
path = Path.cwd()
file = pd.read_excel(path/'keyword.xlsx',engine='openpyxl')
category_list = file.values.tolist()

In [53]:
column = ['업무','공고번호-차수','분류','공고명','공고기관','수요기관','계약방법','입력일시','입찰마감일시']
df = pd.DataFrame(columns=column)
keywords = []
try:
    driver = webdriver.Chrome()
    driver.implicitly_wait(2)

    for query in category_list:

        
        
        driver.get('https://www.g2b.go.kr:8101/ep/tbid/tbidFwd.do')
        task_dict = {'용역': 'taskClCds5'}
        for task in task_dict.values():
            checkbox = driver.find_element(By.ID, task)
            checkbox.click()
        # ID값이 bidNm인 태그 가져오기
        bidNm = driver.find_element(By.ID, 'bidNm')
        # 내용을 삭제
        bidNm.clear()
        # 검색어에 문자열 전달
        bidNm.send_keys(query)
        bidNm.send_keys(Keys.RETURN)
        
        option_dict = {'검색기간 1달': 'setMonth1_1', '입찰마감건 제외': 'exceptEnd', '검색건수 표시': 'useTotalCount'}
        for option in option_dict.values():
            checkbox = driver.find_element(By.ID, option)
            checkbox.click()

        # 목록수 100건 선택 (드롭다운)
        print(1)
        recordcountperpage = driver.find_element(By.NAME,'recordCountPerPage')
        selector = Select(recordcountperpage)
        selector.select_by_value('100')

        search_button = driver.find_element(By.CLASS_NAME, 'btn_mdl')
        search_button.click()
        print(2)
        # 검색 결과 확인
        elem = driver.find_element(By.CLASS_NAME,'results')
        div_list = elem.find_elements(By.TAG_NAME, 'tr')

        # 리스트 형태로 검색결과 저장
        
        print(3)
        results = []
        links = []
        for div in tqdm(div_list):
            results.append(div.text)
            row_data = [div.text.split('\n')]
            a_tags = div.find_elements(By.TAG_NAME,'a')
            if a_tags:
                a_tag = a_tags[0]
                link = a_tag.get_attribute('href')
                row_data.append([link])
                keywords.append(query)
            # create a new DataFrame with the data from this row
            
            df_row = pd.DataFrame(row_data)
            # append it to the overall df using pd.concat()
            df = pd.concat([df, df_row], ignore_index=True)
                #df = df.assign(링크 =links)

except Exception as e:
    print(e)
finally:  
    driver.quit()

1
2
3


100%|██████████| 8/8 [00:02<00:00,  2.97it/s]


1
2
3


100%|██████████| 17/17 [00:03<00:00,  4.91it/s]


In [54]:
df = df[~df[0].isin(['업무 공고번호-차수 분류 공고명 공고기관 수요기관 계약방법 입력일시','검색된 데이터가 없습니다.'])]

In [56]:
column = ['업무','공고번호-차수','분류','공고명','공고기관','수요기관','계약방법','입력일시','입찰마감일시']
new_df = df.iloc[:,9:19]
new_df.columns = column

In [58]:
flatten_keyword = [item for sublist in keywords for item in sublist]

['데이터 분석', '데이터 분석', '데이터 분석', '데이터 분석', '데이터 분석', '데이터 분석', '데이터 분석', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계', '회계']


In [59]:
aa = new_df.iloc[::2,:]
with_link = pd.DataFrame(new_df.iloc[1::2,:]['업무'])
with_link = list(with_link['업무'])
aa['링크'] = with_link
aa.insert(0,'키워드',flatten_keyword)


C:\Users\JOONHY~1\AppData\Local\Temp/ipykernel_45492/2250526200.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['링크'] = with_link


In [61]:
aa['입력일시'] = pd.to_datetime(aa['입력일시'])
aa = aa.sort_values('입력일시',ascending=False)
aa = aa.drop_duplicates(['공고번호-차수'])
aa = aa.reset_index(drop=True) 


C:\Users\JOONHY~1\AppData\Local\Temp/ipykernel_45492/1738884339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['입력일시'] = pd.to_datetime(aa['입력일시'])


In [63]:
from datetime import datetime 
today = datetime.today().strftime('%Y-%m-%d')

In [64]:
aa.to_excel(f'RA_Operational_공고_{today}.xlsx')

In [42]:
aa

,키워드,업무,공고번호-차수,분류,공고명,공고기관,수요기관,계약방법,입력일시,입찰마감일시,링크
0,[회계],용역,20230720087-00,일반,2023-2025 회계연도 회계감사 용역 (재공고),한국해양수산연수원,한국해양수산연수원,제한(총액)협상에의한계약,2023-07-14 17:49:00,(2023/07/25 10:00),https://www.g2b.go.kr:8081/ep/invitation/publi...
1,[회계],용역,20230720155-00,일반,2023~2025년도 회계감사 용역,한국세라믹기술원,한국세라믹기술원,제한(총액)협상에의한계약,2023-07-14 16:06:00,(2023/07/25 15:10),https://www.g2b.go.kr:8081/ep/invitation/publi...
2,[회계],용역,20230719837-00,일반,농어업재해재보험기금 회계처리 기준 정립을 위한 연구용역,농업정책보험금융원,농업정책보험금융원,제한(총액)협상에의한계약,2023-07-14 14:22:00,(2023/07/24 15:00),https://www.g2b.go.kr:8081/ep/invitation/publi...
3,[회계],용역,20230719716-00,일반,재무회계 성능개선을 위한 페이퍼리스 고도화,한국건설기술연구원,한국건설기술연구원,제한(총액)협상에의한계약,2023-07-14 13:53:00,(2023/07/25 11:00),https://www.g2b.go.kr:8081/ep/invitation/publi...
4,[회계],용역,20230717897-00,긴급,2023년도 주택도시보증공사 채권상각·매각 회계자문용역,조달청 부산지방조달청,주택도시보증공사,일반(총액)협상에의한계약,2023-07-13 15:46:00,(2023/08/01 10:00),https://www.g2b.go.kr:8081/ep/invitation/publi...
5,[회계],용역,2023071305001-00,일반,'23년 회계자문용역,한국가스공사,한국가스공사,제한(총액),2023-07-13 09:58:00,(2023/08/23 15:30),https://www.g2b.go.kr:8101/ep/tbid/serviceBidD...
6,[회계],용역,20230717215-00,일반,전략경영지원실(재무회계팀) 육아휴직 대체인력 파견용역,재단법인 경북테크노파크,재단법인 경북테크노파크,수의(총액)소액수의,2023-07-13 09:34:00,(2023/07/19 14:00),https://www.g2b.go.kr:8081/ep/invitation/publi...
7,[회계],용역,20230716399-00,일반,울산광역시 하수도사업특별회계 자산 재평가 용역 (재공고),울산광역시,울산광역시,수의(총액)소액-견적입찰(2인 이상 견적 제출),2023-07-13 08:57:00,(2023/07/19 10:00),https://www.g2b.go.kr:8081/ep/invitation/publi...
8,[데이터 분석],용역,202306743-00,일반,[긴급]데이터 분석 플랫폼 구축,한국도로공사,한국도로공사 본사,제한(총액),2023-07-13 00:00:00,(2023/07/25 10:00),https://www.g2b.go.kr:8101/ep/tbid/serviceBidD...
9,[회계],용역,20230714948-00,일반,2023년도 수도사업특별회계 회계감사 용역,서울특별시 상수도사업본부,서울특별시 상수도사업본부,수의(총액)소액-견적입찰(2인 이상 견적 제출),2023-07-11 19:22:00,(2023/07/17 12:00),https://www.g2b.go.kr:8081/ep/invitation/publi...


In [13]:
import win32com.client as win32
from datetime import date
outlook = win32.gencache.EnsureDispatch('Outlook.Application')

# 새 메일 쓰기창 열기
new_mail = outlook.CreateItem(0)
new_mail.Subject = f"{date.today().strftime('%Y-%m-%d 나라장터 입찰 리스트')}"

#수신자
to_list = ['josuh@deloitte.com']
new_mail.To = ";".join(to_list)
#cc_list = ['yosohn@deloitte.com']
#new_mail.CC = ";".join(cc_list)
attachment1 = rf'C:\Users\josuh\Desktop\나라장터 크롤링\autocrawling\RA_Operational_공고_{today}.xlsx'
new_mail.Attachments.Add(Source=attachment1)


new_mail.Send()


# 아웃룩 종료
outlook.Quit()

com_error: (-2147467260, '작업이 중단되었습니다.', None, None)